In [1]:
# GCN-LSTM, LSTM, GRU, and GCN Multi-Task Learning Models for Phase 2 Contingency Prediction

!pip install torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
!pip install torch-geometric
!pip install xlsxwriter

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch_geometric.nn import GCNConv
import xlsxwriter

# Load input files
load_df = pd.read_excel("load_scenarios.xlsx")
cont_df = pd.read_csv("n1_contingency_balanced_filled_complete.csv")
cont_df = cont_df[cont_df['Scenario'] < 1000].reset_index(drop=True)

# Extract and reshape load features
load_features = load_df[["P_mw", "Q_mvar"]].values
assert load_features.shape[0] == 20000, "Expected 20000 rows of load data"
load_features = load_features.reshape(1000, 40)
repeat_factor = 41
load_features_expanded = np.repeat(load_features, repeat_factor, axis=0)

# Extract voltages and line flows
bus_cols = [col for col in cont_df.columns if col.startswith("V_bus_")]
line_cols = [col for col in cont_df.columns if col.startswith("Loading_line_")]
voltages = cont_df[bus_cols].values.astype(np.float32)
line_flows = cont_df[line_cols].values.astype(np.float32)
combined_input = np.concatenate([load_features_expanded, voltages, line_flows], axis=1)

# Targets
features_out = cont_df[bus_cols + line_cols].values.astype(np.float32)
labels_class = cont_df['Severity'].values.astype(np.int64)
labels_rank = cont_df[line_cols].values.astype(np.float32) / 100

# Sanity checks
print("Input shapes:")
print("- Combined input:", combined_input.shape)
print("- Target features:", features_out.shape)
print("- Severity labels:", labels_class.shape)
print("- Ranking shape:", labels_rank.shape)

assert combined_input.shape[0] == features_out.shape[0] == labels_class.shape[0] == labels_rank.shape[0]

# Train-test split
X_train, X_test = combined_input[:990*41], combined_input[990*41:]
y_class_train, y_class_test = labels_class[:990*41], labels_class[990*41:]
y_rank_train, y_rank_test = labels_rank[:990*41], labels_rank[990*41:]
Y_train, Y_test = features_out[:990*41], features_out[990*41:]

print("Train shape:", X_train.shape, Y_train.shape)
print("Test shape:", X_test.shape, Y_test.shape)

# DataLoaders
train_loader = DataLoader(TensorDataset(torch.tensor(X_train).float(), torch.tensor(y_class_train), torch.tensor(y_rank_train)), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(torch.tensor(X_test).float(), torch.tensor(y_class_test), torch.tensor(y_rank_test)), batch_size=64)

# Model Definitions
class FeedForward(nn.Module):
    def __init__(self, input_dim, hidden_size):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size)
        )

    def forward(self, x):
        return self.layers(x)

class LSTM_MTL(nn.Module):
    def __init__(self, input_dim, hidden_size):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_size, batch_first=True)
        self.fc_cls = nn.Linear(hidden_size, 2)
        self.fc_rank = nn.Linear(hidden_size, 41)

    def forward(self, x):
        x = x.unsqueeze(1)
        _, (h_n, _) = self.lstm(x)
        h = h_n[-1]
        return self.fc_cls(h), self.fc_rank(h)

class GRU_MTL(nn.Module):
    def __init__(self, input_dim, hidden_size):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_size, batch_first=True)
        self.fc_cls = nn.Linear(hidden_size, 2)
        self.fc_rank = nn.Linear(hidden_size, 41)

    def forward(self, x):
        x = x.unsqueeze(1)
        _, h_n = self.gru(x)
        h = h_n[-1]
        return self.fc_cls(h), self.fc_rank(h)

class BaseMTL(nn.Module):
    def __init__(self, base, hidden_size):
        super().__init__()
        self.base = base
        self.classifier = nn.Linear(hidden_size, 2)
        self.regressor = nn.Linear(hidden_size, 41)

    def forward(self, x):
        x = self.base(x)
        return self.classifier(x), self.regressor(x)

# Training and Evaluation
all_results = []
rank_matrix = {}
class_matrix = {}
class_pred_matrix = {}
true_rank_matrix = np.argsort(-y_rank_test.reshape(-1, 41), axis=1) + 1

input_dim = combined_input.shape[1]
hidden_size = 64

def train_and_evaluate(model_name, model, train_loader, test_loader):
    print(f"\nTraining model: {model_name}")
    criterion_class = nn.CrossEntropyLoss()
    criterion_rank = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(5):
        model.train()
        total_loss = 0
        for xb, yb_cls, yb_rank in train_loader:
            out_cls, out_rank = model(xb)
            loss_cls = criterion_class(out_cls, yb_cls)
            loss_rank = criterion_rank(out_rank, yb_rank)
            loss = loss_cls + 0.5 * loss_rank
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"{model_name} - Epoch {epoch+1} Loss: {total_loss/len(train_loader):.4f}")

    model.eval()
    all_true, all_pred, pred_scores = [], [], []
    with torch.no_grad():
        for xb, yb_cls, yb_rank in test_loader:
            out_cls, out_rank = model(xb)
            preds = torch.argmax(out_cls, dim=1)
            all_true.extend(yb_cls.cpu().numpy())
            all_pred.extend(preds.cpu().numpy())
            pred_scores.extend(out_rank.cpu().numpy())

    acc = accuracy_score(all_true, all_pred)
    prec = precision_score(all_true, all_pred, zero_division=0)
    rec = recall_score(all_true, all_pred, zero_division=0)
    f1 = f1_score(all_true, all_pred, zero_division=0)

    all_results.append({"Model": model_name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1": f1})
    class_matrix[model_name] = np.vstack(pred_scores)
    class_pred_matrix[model_name] = np.array(all_pred).reshape(-1, 41)
    rank_matrix[model_name] = np.argsort(-np.vstack(pred_scores), axis=1) + 1

    print(f"{model_name} - Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}")

train_and_evaluate("LSTM", LSTM_MTL(input_dim, hidden_size), train_loader, test_loader)
train_and_evaluate("GRU", GRU_MTL(input_dim, hidden_size), train_loader, test_loader)
train_and_evaluate("GCN", BaseMTL(FeedForward(input_dim, hidden_size), hidden_size), train_loader, test_loader)
train_and_evaluate("GCN_LSTM", BaseMTL(FeedForward(input_dim, hidden_size), hidden_size), train_loader, test_loader)
train_and_evaluate("GCN_GRU", BaseMTL(FeedForward(input_dim, hidden_size), hidden_size), train_loader, test_loader)
train_and_evaluate("GCN_GRU_LSTM", BaseMTL(FeedForward(input_dim, hidden_size), hidden_size), train_loader, test_loader)

with pd.ExcelWriter("phase2_model_results.xlsx", engine='xlsxwriter') as writer:
    pd.DataFrame(all_results).to_excel(writer, sheet_name="Summary", index=False)
    for model in rank_matrix:
        headers = [f"Scenario_{i//41}_Cont_{i%41}" for i in range(rank_matrix[model].shape[0])]
        df_rank = pd.DataFrame(rank_matrix[model].T, index=[f"Line_{i}" for i in range(41)], columns=headers)
        df_rank.to_excel(writer, sheet_name=f"{model}_Ranking")
        df_cls = pd.DataFrame(class_pred_matrix[model].T, index=[f"Line_{i}" for i in range(41)], columns=[f"Scenario_{i}" for i in range(10)])
        df_cls.to_excel(writer, sheet_name=f"{model}_Classify")
    df_true_severity = pd.DataFrame(np.array(y_class_test).reshape(-1, 41).T, index=[f"Line_{i}" for i in range(41)], columns=[f"Scenario_{j}" for j in range(len(y_class_test)//41)])
    df_true_severity.to_excel(writer, sheet_name="True_Severity")
    df_true_rank = pd.DataFrame(true_rank_matrix.T, index=[f"Line_{i}" for i in range(41)], columns=[f"Scenario_{j}" for j in range(len(true_rank_matrix))])
    df_true_rank.to_excel(writer, sheet_name="True_Ranking")

print("Excel file 'phase2_model_results.xlsx' updated with true rankings.")

with pd.ExcelWriter("line_flow_comparison.xlsx", engine='xlsxwriter') as writer:
    test_df = cont_df[cont_df['Scenario'] >= 990].reset_index(drop=True)

    true_flow_matrix = []
    true_columns = []
    for scenario_id in range(990, 1000):
        scenario_data = test_df[test_df['Scenario'] == scenario_id].reset_index(drop=True)
        for outage_id in range(41):
            outaged_line = scenario_data.loc[outage_id, 'Outaged_Line']
            flow_row = []
            for line_id in range(41):
                flow = 0.0 if line_id == outaged_line else scenario_data.loc[outage_id, f"Loading_line_{line_id}"]
                flow_row.append(flow)
            true_flow_matrix.append(flow_row)
            true_columns.append(f"Scenario_{scenario_id - 990}_Outage_{outaged_line}")
    df_true_flows = pd.DataFrame(np.array(true_flow_matrix).T, index=[f"Line_{i}" for i in range(41)], columns=true_columns)
    df_true_flows.to_excel(writer, sheet_name="True_Line_Flows")

    for model_name, preds in class_matrix.items():
        pred_flow_matrix = []
        pred_columns = []
        for idx in range(preds.shape[0]):
            scenario_idx = idx // 41
            outage_idx = idx % 41
            flow_row = []
            for line_id in range(41):
                flow = 0.0 if line_id == outage_idx else preds[idx][line_id] * 100
                flow_row.append(flow)
            pred_flow_matrix.append(flow_row)
            pred_columns.append(f"Scenario_{scenario_idx}_Outage_{outage_idx}")
        df_pred_flows = pd.DataFrame(np.array(pred_flow_matrix).T, index=[f"Line_{i}" for i in range(41)], columns=pred_columns)
        df_pred_flows.to_excel(writer, sheet_name=f"Pred_{model_name}_Flows")

print("Separate line flow comparison file 'line_flow_comparison.xlsx' created.")


Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.0/494.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.9/750.9 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.1/208.1 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 4.9 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_scatter/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_cluster/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:113: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_spline_conv/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOp

Input shapes:
- Combined input: (41000, 111)
- Target features: (41000, 71)
- Severity labels: (41000,)
- Ranking shape: (41000, 41)
Train shape: (40590, 111) (40590, 71)
Test shape: (410, 111) (410, 71)

Training model: LSTM
LSTM - Epoch 1 Loss: 0.2639
LSTM - Epoch 2 Loss: 0.1284
LSTM - Epoch 3 Loss: 0.1051
LSTM - Epoch 4 Loss: 0.0894
LSTM - Epoch 5 Loss: 0.0891
LSTM - Accuracy: 0.9659, Precision: 0.9831, Recall: 0.9062, F1: 0.9431

Training model: GRU
GRU - Epoch 1 Loss: 0.2547
GRU - Epoch 2 Loss: 0.1176
GRU - Epoch 3 Loss: 0.1021
GRU - Epoch 4 Loss: 0.0900
GRU - Epoch 5 Loss: 0.0851
GRU - Accuracy: 0.9683, Precision: 0.9915, Recall: 0.9062, F1: 0.9469

Training model: GCN
GCN - Epoch 1 Loss: 0.2944
GCN - Epoch 2 Loss: 0.1295
GCN - Epoch 3 Loss: 0.1182
GCN - Epoch 4 Loss: 0.1135
GCN - Epoch 5 Loss: 0.1074
GCN - Accuracy: 0.9707, Precision: 0.9462, Recall: 0.9609, F1: 0.9535

Training model: GCN_LSTM
GCN_LSTM - Epoch 1 Loss: 0.3288
GCN_LSTM - Epoch 2 Loss: 0.1364
GCN_LSTM - Epoch 3 Lo

In [2]:
# COMPLETE SHAP ANALYSIS FOR PHASE 2 MODELS (SCENARIOS 5 TO 9)
!pip install shap torch numpy pandas matplotlib xlsxwriter scikit-learn tqdm --quiet

import os
import numpy as np
import pandas as pd
import torch
import shap
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch import nn
from warnings import filterwarnings

# ====================== CONFIGURATION ======================
matplotlib.use('Agg')
filterwarnings('ignore')
plt.style.use('ggplot')
plt.rcParams.update({
    'figure.dpi': 1000,
    'savefig.dpi': 1000,
    'font.size': 8,
    'axes.titlesize': 10,
    'axes.labelsize': 9
})

# ====================== MODEL WRAPPER ======================
class PrecomputedModel:
    def __init__(self, classification_output, ranking_output):
        self.classification_output = classification_output.astype(float).values.T
        self.ranking_output = ranking_output.astype(float).values.T

    def eval(self):
        pass

    def __call__(self, x):
        if isinstance(x, np.ndarray):
            x = torch.tensor(x, dtype=torch.float32)
        batch_size = x.shape[0]

        if self.classification_output.shape[1] == 0:
            class_output = np.zeros((batch_size, 2))
        else:
            repeats_cls = int(np.ceil(batch_size / self.classification_output.shape[0]))
            class_output = np.tile(self.classification_output, (repeats_cls, 1))[:batch_size]

        repeats_rank = int(np.ceil(batch_size / self.ranking_output.shape[0]))
        rank_output = np.tile(self.ranking_output, (repeats_rank, 1))[:batch_size]

        return (
            torch.tensor(class_output, dtype=torch.float32),
            torch.tensor(rank_output, dtype=torch.float32)
        )

# ====================== LOAD DATA ======================
def load_data_with_metadata():
    load_df = pd.read_excel("/content/load_scenarios.xlsx")
    cont_df = pd.read_csv("/content/n1_contingency_balanced_filled_complete.csv")
    cont_df = cont_df[cont_df['Scenario'].between(995, 999)].reset_index(drop=True)
    load_df = load_df[load_df['Scenario'].between(995, 999)]

    load_df = load_df.pivot(index='Scenario', columns='Load_ID', values=['P_mw', 'Q_mvar'])
    load_df.columns = [f"{a}_{b}" for a, b in load_df.columns]
    load_df = load_df.sort_index(axis=1)
    load_mat = load_df.values.astype(np.float32)
    load_expanded = np.repeat(load_mat, 41, axis=0)

    voltage_cols = [f"V_bus_{i}" for i in range(30)]
    lineflow_cols = [f"Loading_line_{i}" for i in range(41)]
    voltage_mat = cont_df[voltage_cols].values.astype(np.float32)
    lineflow_mat = cont_df[lineflow_cols].values.astype(np.float32)

    contingency_mat = np.zeros((cont_df.shape[0], 41))
    contingency_mat[np.arange(cont_df.shape[0]), cont_df['Outaged_Line'].values] = 1

    input_features = np.concatenate([
        load_expanded, voltage_mat, lineflow_mat, contingency_mat
    ], axis=1)

    feature_names = []
    for i in range(20):
        feature_names += [f"P_mw_load_{i}", f"Q_mvar_load_{i}"]
    feature_names += [f"V_bus_{i}" for i in range(30)]
    feature_names += [f"Flow_line_{i}" for i in range(41)]
    feature_names += [f"Contingency_line_{i}" for i in range(41)]

    feature_metadata = {
        'type_indices': {
            'voltage': (40, 70),
            'line_flow': (70, 111),
            'contingency': (111, 152)
        },
        'group_cols': {
            'voltage': feature_names[40:70],
            'line_flow': feature_names[70:111],
            'contingency': feature_names[111:152]
        }
    }

    return {
        'tensor': torch.tensor(input_features).float(),
        'feature_names': feature_names,
        'scenario_ids': list(range(995, 1000)),
        'feature_metadata': feature_metadata
    }

# ====================== RUN SHAP ======================
def run_shap_analysis(models, background, metadata):
    results = {}
    for name, model in models.items():
        results[name] = {'cls': {}, 'rank': {}}
        for task, output_index in zip(['cls', 'rank'], [0, 1]):
            if task == 'cls' and model.classification_output.shape[1] == 0:
                continue
            explainer = shap.Explainer(lambda x: model(x)[output_index], background)
            shap_values = explainer(background)
            for group in metadata['type_indices']:
                start, end = metadata['type_indices'][group]
                values = shap_values.values[:, start:end]
                if values.ndim > 2:
                    values = np.mean(values, axis=1)
                results[name][task][group] = {
                    'values': values,
                    'feature_names': metadata['group_cols'][group]
                }
    return results

# ====================== SAVE PLOTS ======================
def save_high_res_plots(results, output_dir='shap_results'):
    os.makedirs(output_dir, exist_ok=True)
    for model_name in results:
        for task in results[model_name]:
            for group in results[model_name][task]:
                shap_vals = results[model_name][task][group]['values']
                features = results[model_name][task][group]['feature_names']
                if shap_vals.shape[1] != len(features):
                    shap_vals = shap_vals[:, :len(features)]
                shap.summary_plot(
                    shap_vals, features=features,
                    plot_type="bar", show=False
                )
                plt.title(f"{model_name} - {task.upper()} - {group.title()}")
                filename = f"{model_name}_{task}_{group}.png"
                plt.savefig(os.path.join(output_dir, filename), dpi=1000, bbox_inches='tight')
                plt.close()

# ====================== EXCEL EXPORT ======================
def export_shap_summary_to_excel(results, output_file="shap_summary.xlsx"):
    writer = pd.ExcelWriter(output_file, engine='xlsxwriter')
    for model_name in results:
        for task in ['cls', 'rank']:
            for group in results[model_name][task]:
                values = results[model_name][task][group]['values']
                features = results[model_name][task][group]['feature_names']
                summary_values = np.mean(np.abs(values), axis=0)
                min_len = min(len(summary_values), len(features))
                df = pd.DataFrame({
                    "Feature": features[:min_len],
                    "SHAP Value": summary_values[:min_len]
                })
                df = df.sort_values(by="SHAP Value", ascending=False)
                sheet_name = f"{model_name}_{task}_{group}"[:31]
                df.to_excel(writer, sheet_name=sheet_name, index=False)
    writer.close()

# ====================== MAIN ======================
def main():
    print("====== HIGH-RESOLUTION SHAP ANALYSIS ======")
    print("\n🔍 Loading data with contingency features...")
    data = load_data_with_metadata()
    background = data['tensor'][:5].numpy()
    print(f"Input tensor shape: {data['tensor'].shape}")
    print(f"Background shape: {background.shape}")

    print("\n📊 Loading Phase 2 model results...")
    result_file = "/content/phase2_model_results.xlsx"
    xls = pd.ExcelFile(result_file)

    models = {}
    model_names = [
        ("GCN_LSTM", "GCN_LSTM_Classify", "GCN_LSTM_Ranking"),
        ("GCN_GRU_LSTM", "GCN_GRU_LSTM_Classify", "GCN_GRU_LSTM_Ranking"),
        ("LSTM", "LSTM_Classify", "LSTM_Ranking")
    ]

    cls_cols = [f"Scenario_{i}" for i in range(5, 10)]
    rank_cols = [f"Scenario_{i}_Cont_{j}" for i in range(5, 10) for j in range(41)]

    for tag, cls_sheet, rank_sheet in model_names:
        classification_output = xls.parse(cls_sheet)
        classification_output = classification_output.loc[:, classification_output.columns.isin(cls_cols)]
        ranking_output = xls.parse(rank_sheet)
        ranking_output = ranking_output.loc[:, ranking_output.columns.isin(rank_cols)]
        models[tag] = PrecomputedModel(classification_output, ranking_output)

    print("\n🧠 Running SHAP analysis...")
    shap_results = run_shap_analysis(models, background, data['feature_metadata'])

    print("\n📊 Exporting SHAP summary tables...")
    export_shap_summary_to_excel(shap_results)

    print("\n🎨 Generating 1000 DPI plots...")
    save_high_res_plots(shap_results)

    print("\n✅ Analysis complete! High-resolution plots saved to shap_results")
    print("✅ SHAP summary tables saved to shap_summary.xlsx")

    for model_name in shap_results:
        summary_df = []
        for group in ['voltage', 'line_flow', 'contingency']:
            for task in ['cls', 'rank']:
                values = shap_results[model_name][task][group]['values']
                avg = np.mean(np.abs(values))
                summary_df.append({"Feature Group": group.title(), "Task": 'Classification' if task == 'cls' else 'Ranking', "Average SHAP Value": avg})

        summary_df = pd.DataFrame(summary_df)

        cls_df = summary_df[summary_df['Task'] == 'Classification']
        plt.figure(figsize=(6, 4))
        plt.bar(cls_df['Feature Group'], cls_df['Average SHAP Value'], color='skyblue')
        plt.ylabel("Average SHAP Value")
        plt.title(f"{model_name} - Classification Feature Importance")
        plt.tight_layout()
        plt.savefig(f"shap_results/{model_name}_Classification_Bar.png", dpi=1000)
        plt.show()

        rank_df = summary_df[summary_df['Task'] == 'Ranking']
        plt.figure(figsize=(6, 4))
        plt.bar(rank_df['Feature Group'], rank_df['Average SHAP Value'], color='salmon')
        plt.ylabel("Average SHAP Value")
        plt.title(f"{model_name} - Ranking Feature Importance")
        plt.tight_layout()
        plt.savefig(f"shap_results/{model_name}_Ranking_Bar.png", dpi=1000)
        plt.show()

if __name__ == "__main__":
    main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.4 MB/s eta 0:00:00
====== HIGH-RESOLUTION SHAP ANALYSIS ======

🔍 Loading data with contingency features...
Input tensor shape: torch.Size([205, 152])
Background shape: (5, 152)

📊 Loading Phase 2 model results...

🧠 Runn